# Лекция 2: Nullable Типы

## Billion Dollar Mistake

```c#
void DoSomething(SomeClass arg) 
{
    // Делаем что-то до этого
    arg.DoSomething(); // ???
    // Что-то делаем после
}
```

Что может произойти не так?

<details>
<summary> Подумойте!!! </summary>

```c#
void DoSomething(SomeClass arg) 
{
    // Делаем что-то до этого
    arg.DoSomething(); // Possible Null Reference Exception (NRE)
    // Что-то делаем после
}
```

Естественно это можно починить следующим образом

```c#
void DoSomething(SomeClass arg) 
{
    // Сообщаем о том, что arg никогда не должен быть null
    // Если срабатывает, то это значит, что инвариант нарушен.
    Assert.NotNull(arg, "Arguement is null!!!");

    if (arg == null)
    {
        // Можем сообщение в какой-нибудь лог
        return;
    }
    // Делаем что-то до этого
    arg.DoSomething(); // Possible Null Reference Exception (NRE)
    // Что-то делаем после
}
```

</details>


## В чем проблема подхода выше ??

<details>
<summary> Снова подумойте!!! </summary>

* Каждый раз писать проверки на null
* Не работает в рантайме, когда хотелось бы в compile time

Большая проблема, у нас все reference type неявно могут быть null

</details>

Проблема в том, что мы не ожидаем аргументом null, а возможности предотвратить этого у нас как будто нет.\
Это приводит к трудно уловимым ошибкам, из-за чего в программе могут появиться неприятные баги.

## Как решали в других языках

### C++: Optional

```c++
std::optional<std::string> GetGuildName(
  const Character& Character) {

  if (Character.hasGuild) {
    return Character.GuildName;
  }

  return std::nullopt;
}

...

int main(){
  Character player;
  auto name = GetGuildName(player);

  std::cout << "Guild: "
    << name.value_or("[None]");
}
```

У нас есть возможность показать явным образом, что значение не инициализировано.

## Как сделано в .net



Были добавленны nullable types

## Value Types

```c#
int? value = null;

if (someFlag)
{
    value = 5;
}

Console.WriteLine(value); // 5 если someFlag == true, иначе ничего
```

Возвращаемое Nullable значение

```c#
public int F(bool someFlag) // Error
{
    int? value = null;

    if (someFlag)
    {
        value = 5;
    }

    return value;
}

...

public int? F(bool someFlag) // Ok
{
    int? value = null;

    if (someFlag)
    {
        value = 5;
    }

    return value;
}
```



## Как работает под капотом

```c#
int? ~= Nullable<int>

int? F(bool? someFlag) // Ok
{
    int? value1 = null;

    if (someFlag.HasValue)
    {
        value1 = 5;
    }

    return value1;
}

На этапе верхнеуровневой компиляции превращается в:

Nullable<int> F(Nullable<bool> someFlag)
{
    Nullable<int> result = null;

    if (someFlag.HasValue)
    {
        result = 5;
    }
    
    return result;
}
```

## Nullable Reference Types

В отличие от value types, reference types могут принимать значение null, причем неявно. \
Это достовляло большой дискомфорт, и поэтому появились Nullable Reference Types как в некотором роде замена Optional

```c#
public record Person(int Age, string Name);

public class Novokuznetsk
{
    public Person? CreateNewPerson()
    {
        if (Random.Shared.Next(0, 2) == 1)
        {
            return new Person(Random.Shared.Next(0, 50), 
                $"Id{Random.Shared.Next(0, 10000)}");
        }

        return null;
    }
}

public class KuznetskMetalZavod
{
    public void GiveJobTo(Person person)
    {
        if (person.Age > 18)
        {
            Console.WriteLine($"Person with name {person.Name} started working");
        }
        else
        {
            Console.WriteLine($"Too smol to work");
        }
    }
}

...

var zavod = new KuznetskMetalZavod();
var kuznya = new Novokuznetsk();

Person person = kuznya.CreateNewPerson(); // Предположим, что получили это из метода, 
                                          // который может вернуть интересный результат
zavod.GiveJob(person);                    // Что будет?        
```

## Roslyn Analyzers или как нам помогают писать код
```c#
zavod.GiveJobTo(person); // Possible null reference argument for parameter 'person' in 'KuznetskMetalZavod.GiveJobTo'
```

## Как работать в данной ситуации

* Поменять сам метод:

```c#
public void GiveJobTo(Person? person)
{
    if (person == null)
    {
        Console.WriteLine("Trying to give never existing person");
        return;
    }

    if (person.Age > 18)
    {
        Console.WriteLine($"Person with name {person.Name} started working");
    }
    else
    {
        Console.WriteLine($"Too smol to work");
    }
}
```



* Либо же поменять код в месте вызова:

```c#
var zavod = new KuznetskMetalZavod();
var kuznya = new Novokuznetsk();

Person person = kuznya.CreateNewPerson();
                                         
if (person == null)
{
    return;
}

zavod.GiveJob(person);                   
```

## Операторы

### Оператор ?.

Если в цепочке операций условного доступа к элементу или члену одна из \
операций возвращает значение null, то следующие за ней операции не выполняются.
Сама цепочка операций вернет null.
```c#
Person person = kuznya.CreateNewPerson();

Console.WriteLine(person.Age)  // Возможно NRE
Console.WriteLine(person?.Age) // НЕ БУДЕТ NRE
```

### Оператор ??

Если в цепочке операций условного доступа к элементу или члену одна из \
операций возвращает значение null, то за место null используется пользовательское значение.
```c#
Person person = kuznya.CreateNewPerson();

Console.WriteLine(person.Age)               // Возможно NRE
Console.WriteLine(person?.Age ?? "No Name") // НЕ БУДЕТ NRE
```

### Оператор !

Не смотря на то, что объект может быть null, мы можем явным образом показать, что выражение \
не может быть null.

```c#
Console.WriteLine(person!.Age)  // Возможно NRE, но это уже на нашей совести
```